In [1]:
from beir.datasets.data_loader import GenericDataLoader

/dss/dsshome1/07/ra65bex2/srawat/myenv/lib/python3.12/site-packages/beir/datasets/data_loader.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
data_path = '/dss/dsshome1/07/ra65bex2/srawat/scidocs/scidocs'

In [3]:
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")

  0%|          | 0/25657 [00:00<?, ?it/s]

In [4]:
contrastive_pairs=[]
import random
c=0
for query_id, relevant_docs in qrels.items():
    try:
        query_text = queries[query_id]
        for doc_id in relevant_docs:
            positive = corpus[doc_id]["text"]
        #print(relevant_docs)
        positive_doc_ids = set(relevant_docs)
        all_doc_ids = set(corpus.keys())
        negative_doc_ids = all_doc_ids - positive_doc_ids
        negative_doc_ids=list(negative_doc_ids)
        negative_doc_samples = random.sample(negative_doc_ids, k=5)
        negatives=[]
        for neg_doc_id in negative_doc_samples:
            negative_doc_text = corpus[neg_doc_id]["text"]
            negatives.append(negative_doc_text)
        contrastive_pairs.append({
            "anchor": query_text,
            "positive": positive,
            "negatives": negatives
        })
    except:
        c=c+1

In [5]:
c

0

In [6]:
len(contrastive_pairs)

1000

In [7]:
len(qrels)

1000

In [8]:
from torch.utils.data import DataLoader

In [9]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [10]:
contrastive_dataset = ContrastiveDataset(contrastive_pairs)

In [11]:
data_loader = DataLoader(contrastive_dataset, batch_size=32, shuffle=True)

In [12]:
len(data_loader)

32

In [13]:
import torch
file_path="/dss/dsshome1/07/ra65bex2/srawat/contrastive_learning/v1.1/0.1hyperbolic/hyperbolic_lora_checkpoint_epoch_3.pth"
lora_model = torch.load(file_path)

/tmp/ipykernel_2596275/3548975511.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_model = torch.load(file_path)


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lora_model = lora_model.to(device)

In [15]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [16]:
def lorentzian_distance(x, y):
    
    dot_product = torch.sum(x * y, dim=-1)
    norm_x = torch.norm(x, dim=-1)
    norm_y = torch.norm(y, dim=-1)
    
    distance = torch.acosh(-dot_product + torch.sqrt((1 + norm_x**2) * (1 + norm_y**2)))
    return distance

In [17]:
def expm_o(v, c=1.0):
    c = torch.tensor(c)
    vspace = v
    vnorm = torch.norm(v, p=2, dim=-1, keepdim=True)
    xspace = torch.sinh(torch.sqrt(c) * vnorm) * vspace / (torch.sqrt(c) * vnorm)
    batch_min = xspace.min(dim=1, keepdim=True).values
    batch_max = xspace.max(dim=1, keepdim=True).values
    xspace_scaled=(xspace - batch_min) / (batch_max - batch_min)
    return xspace_scaled

In [18]:
def evaluate_mrr(model1, data_loader_val, distance_fn):
    model1.eval()
    
    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = expm_o(model1(**anchor_input).last_hidden_state[:, 0, :])
            positive_embedding = expm_o(model1(**positive_input).last_hidden_state[:, 0, :])
            negative_embedding = [expm_o(model1(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

            pos_dist = distance_fn(anchor_embedding, positive_embedding)
            neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
            all_similarities=torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)
            
    mrr = total_rr / num_queries
    return mrr

In [19]:
mrr_validation = evaluate_mrr(model1=lora_model, data_loader_val=data_loader,distance_fn=lorentzian_distance)
print(mrr_validation)

0.3976000068187714
